# Can I plot the single-cell tracks and glimpses

Using the outliers that are 1sd away from the mean of dMtb

In [1]:
import os, glob
import btrack
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from macrohet import dataio, tools, visualise
from tqdm.auto import tqdm
from scipy import stats
import pickle
import re
import cv2
from skimage import io
colors = sns.set_palette('PiYG')
track_scale_factor = 5.04

### Load metadata

In [2]:
base_dir = '/mnt/DATA/macrohet/'
# base_dir = '/Volumes/lab-gutierrezm/home/users/dayn/macrohet/'
metadata_fn = os.path.join(base_dir, 'macrohet_images/Index.idx.xml')
metadata = dataio.read_harmony_metadata(metadata_fn)  
metadata

Reading metadata XML file...


Extracting HarmonyV5 metadata:   0%|          | 0/113400 [00:00<?, ?it/s]

Extracting metadata complete!


,id,State,URL,Row,Col,FieldID,PlaneID,TimepointID,ChannelID,FlimID,...,PositionZ,AbsPositionZ,MeasurementTimeOffset,AbsTime,MainExcitationWavelength,MainEmissionWavelength,ObjectiveMagnification,ObjectiveNA,ExposureTime,OrientationMatrix
0,0303K1F1P1R1,Ok,r03c03f01p01-ch1sk1fk1fl1.tiff,3,3,1,1,0,1,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
1,0303K1F1P1R2,Ok,r03c03f01p01-ch2sk1fk1fl1.tiff,3,3,1,1,0,2,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
2,0303K1F1P2R1,Ok,r03c03f01p02-ch1sk1fk1fl1.tiff,3,3,1,2,0,1,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
3,0303K1F1P2R2,Ok,r03c03f01p02-ch2sk1fk1fl1.tiff,3,3,1,2,0,2,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
4,0303K1F1P3R1,Ok,r03c03f01p03-ch1sk1fk1fl1.tiff,3,3,1,3,0,1,1,...,4E-06,0.135587499,0,2021-04-16T19:09:34.4+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113395,0609K75F9P1R2,Ok,r06c09f09p01-ch2sk75fk1fl1.tiff,6,9,9,1,74,2,1,...,0,0.135533601,266399.61,2021-04-19T21:14:19.477+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113396,0609K75F9P2R1,Ok,r06c09f09p02-ch1sk75fk1fl1.tiff,6,9,9,2,74,1,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113397,0609K75F9P2R2,Ok,r06c09f09p02-ch2sk75fk1fl1.tiff,6,9,9,2,74,2,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113398,0609K75F9P3R1,Ok,r06c09f09p03-ch1sk75fk1fl1.tiff,6,9,9,3,74,1,1,...,4E-06,0.135537595,266399.61,2021-04-19T21:14:20.037+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."


In [3]:
metadata_path = os.path.join(base_dir, 'macrohet_images/Assaylayout/20210602_Live_cell_IPSDMGFP_ATB.xml')
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,)
assay_layout

Reading metadata XML file...
Extracting metadata complete!


Strain Compound Concentration ConcentrationEC
3 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
4 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
5 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99
6 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99

### Load cells from outlier dMtb analysis

In [4]:
# load glimpses from outliers and extract unique_IDs from them
outlier_fns = glob.glob('/mnt/DATA/macrohet/results/glimpses/interactive_plots/dMtb/*.png')
unique_IDs = [fn.split('_')[2] for fn in outlier_fns]

In [5]:
unique_IDs

['701.6.5',
 '421.3.9',
 '416.5.7',
 '166.4.6',
 '2198.5.9',
 '184.4.5',
 '701.3.4',
 '1138.3.5',
 '1809.3.9',
 '1254.4.5',
 '82.4.7',
 '1578.5.9',
 '609.4.4',
 '535.3.7',
 '114.6.7',
 '91.5.7',
 '70.5.5',
 '510.4.5',
 '2201.3.9',
 '2135.3.4',
 '2144.3.8',
 '78.4.6',
 '1894.6.5',
 '279.6.8',
 '331.3.8',
 '640.4.4',
 '513.6.8',
 '354.5.9',
 '504.5.5',
 '1254.3.6',
 '223.3.5',
 '467.5.5',
 '1118.4.7',
 '904.4.5',
 '320.4.4',
 '839.3.8',
 '1322.5.5',
 '2088.3.7',
 '546.6.4',
 '749.5.7',
 '461.4.8',
 '1805.4.6',
 '182.4.4',
 '1693.5.7',
 '905.3.8',
 '2190.6.6',
 '24.6.9',
 '2135.3.4',
 '1418.6.4',
 '1254.4.5',
 '1472.4.4',
 '78.4.6',
 '2202.3.8',
 '167.4.4',
 '314.4.7',
 '621.4.5',
 '546.6.4',
 '2180.6.5',
 '1404.5.6',
 '150.4.5',
 '897.6.8',
 '659.3.4',
 '705.5.7',
 '360.3.7',
 '902.4.9',
 '362.3.5',
 '1075.6.5',
 '470.5.8',
 '304.4.4',
 '353.6.5',
 '1065.5.4',
 '676.3.9',
 '2549.3.8',
 '688.5.7',
 '235.5.7',
 '143.4.7',
 '1418.6.4',
 '493.6.8',
 '1035.3.9',
 '299.3.6',
 '128.3.5',
 '905.

In [6]:
len(unique_IDs)

664

# Load sc_df

In [7]:
# Specify the path to your pickle file
sc_df_path = "/mnt/DATA/macrohet/results/preliminary_sc_measures/sc_dfs/sc_df_>40_interp.pkl" 
#"/mnt/DATA/macrohet/results/preliminary_sc_measures/sc_dfs/sc_df_>40.pkl"

# Load the pickle file
with open(sc_df_path, "rb") as file:
    df = pickle.load(file)

In [8]:
# subset_df = df[df['Unique_ID'].isin(true_unique_IDs)] # only GT 3.5 tracks
subset_df = df[df['Unique ID'].isin(unique_IDs)] # all outlier tracks for now

In [9]:
subset_df

,Time (hours),x,y,x scaled,y scaled,Infection status,Initial infection status,Final infection status,Area,Intracellular mean Mtb content,...,Eccentricity,MSD,Strain,Compound,Concentration,Cell ID,Acquisition ID,Unique ID,Technical replicate,dMtb
0,0,708.125061,77.589088,3568.950308,391.049006,0.0,0.0,1.0,42556.0,365.408905,...,0.813310,0.000000,RD1,CTRL,EC0,433,"(3, 4)",433.3.4,1.0,0.095232
1,1,725.449097,73.245003,3656.263447,369.154814,0.0,0.0,1.0,48727.0,362.852478,...,0.896474,17.860383,RD1,CTRL,EC0,433,"(3, 4)",433.3.4,1.0,0.095232
2,2,723.088928,76.431496,3644.368198,385.214738,0.0,0.0,1.0,43484.0,363.515442,...,0.876911,3.965367,RD1,CTRL,EC0,433,"(3, 4)",433.3.4,1.0,0.095232
3,3,726.009766,79.395073,3659.089219,400.151168,0.0,0.0,1.0,39113.0,360.448578,...,0.717504,4.161019,RD1,CTRL,EC0,433,"(3, 4)",433.3.4,1.0,0.095232
4,4,728.209656,82.197769,3670.176665,414.276757,0.0,0.0,1.0,32644.0,362.289764,...,0.425791,3.562951,RD1,CTRL,EC0,433,"(3, 4)",433.3.4,1.0,0.095232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
697433,70,565.650940,1081.278931,2850.880737,5449.645811,0.0,1.0,0.0,61587.0,374.370483,...,0.521127,0.963656,WT,BDQ,EC99,1202,"(6, 9)",1202.6.9,2.0,-0.835031
697434,71,566.611755,1083.470947,2855.723247,5460.693574,0.0,1.0,0.0,58778.0,372.559570,...,0.519641,2.393346,WT,BDQ,EC99,1202,"(6, 9)",1202.6.9,2.0,-0.835031
697435,72,568.148987,1081.411011,2863.470894,5450.311494,0.0,1.0,0.0,64893.0,372.814087,...,0.579433,2.570296,WT,BDQ,EC99,1202,"(6, 9)",1202.6.9,2.0,-0.835031
697436,73,570.763306,1084.552124,2876.647061,5466.142705,1.0,1.0,0.0,61917.0,374.565948,...,0.554528,4.086717,WT,BDQ,EC99,1202,"(6, 9)",1202.6.9,2.0,-0.835031


In [10]:
from macrohet import tile, dataio, visualise, aws_tools
from skimage.io import imsave
import os

# Iterate over positions multiprocessing

Prone to crashing after a while 

#### Due to the nature of multiprocessing, it isn't going to be simple to make sure it doesn't have multiple iterations...

In [11]:
import multiprocessing

In [12]:
from macrohet.notify import send_sms

# Original multiprocessing

Prone to crashing after a while

In [13]:
global previous_row, previous_column
previous_row, previous_column = 0,0

# exclude fns which have already been created:

In [19]:
glimpse_dir = '/mnt/DATA/macrohet/results/glimpses/interactive_plots/time_series'
glimpse_fns = glob.glob(os.path.join(glimpse_dir, '*.png'))

In [27]:
glimpse_fns

['/mnt/DATA/macrohet/results/glimpses/interactive_plots/time_series/glimpse_486.3.5_t38.png',
 '/mnt/DATA/macrohet/results/glimpses/interactive_plots/time_series/glimpse_639.4.5_t17.png',
 '/mnt/DATA/macrohet/results/glimpses/interactive_plots/time_series/glimpse_877.5.8_t26.png',
 '/mnt/DATA/macrohet/results/glimpses/interactive_plots/time_series/glimpse_510.4.5_t42.png',
 '/mnt/DATA/macrohet/results/glimpses/interactive_plots/time_series/glimpse_236.6.5_t60.png',
 '/mnt/DATA/macrohet/results/glimpses/interactive_plots/time_series/glimpse_495.6.7_t13.png',
 '/mnt/DATA/macrohet/results/glimpses/interactive_plots/time_series/glimpse_923.3.4_t55.png',
 '/mnt/DATA/macrohet/results/glimpses/interactive_plots/time_series/glimpse_331.3.8_t52.png',
 '/mnt/DATA/macrohet/results/glimpses/interactive_plots/time_series/glimpse_355.4.4_t13.png',
 '/mnt/DATA/macrohet/results/glimpses/interactive_plots/time_series/glimpse_599.3.8_t63.png',
 '/mnt/DATA/macrohet/results/glimpses/interactive_plots/time

In [31]:
already_made_glimpses = list()
for fn in glimpse_fns:
    unique_ID = fn.split('_')[3]
    time = fn.split('_')[-1].replace('t','').replace('.png','')
    glimpse_ID = f'{unique_ID}.{time}'
    already_made_glimpses.append(glimpse_ID)

In [32]:
already_made_glimpses

['486.3.5.38',
 '639.4.5.17',
 '877.5.8.26',
 '510.4.5.42',
 '236.6.5.60',
 '495.6.7.13',
 '923.3.4.55',
 '331.3.8.52',
 '355.4.4.13',
 '599.3.8.63',
 '28.4.7.18',
 '232.4.7.46',
 '849.5.7.52',
 '3.5.9.57',
 '279.6.8.7',
 '50.3.4.35',
 '162.4.5.51',
 '673.3.5.3',
 '437.4.8.15',
 '151.5.4.29',
 '1923.6.4.19',
 '264.3.9.4',
 '24.6.9.19',
 '289.4.9.50',
 '767.6.8.55',
 '1894.6.5.20',
 '276.4.6.4',
 '305.3.7.34',
 '534.3.4.23',
 '2164.5.7.29',
 '1819.3.7.56',
 '640.4.4.19',
 '2067.6.8.36',
 '166.4.6.1',
 '823.3.7.20',
 '1809.3.9.21',
 '62.3.5.37',
 '525.3.7.74',
 '1404.5.6.20',
 '279.6.8.35',
 '323.3.6.67',
 '475.6.9.40',
 '396.5.5.15',
 '393.3.4.26',
 '1134.4.9.29',
 '847.4.6.49',
 '404.5.5.22',
 '1035.3.9.31',
 '421.3.9.0',
 '362.3.5.46',
 '415.4.9.29',
 '106.3.5.24',
 '404.5.5.4',
 '50.3.4.29',
 '1766.4.8.39',
 '892.3.7.29',
 '1653.5.6.37',
 '1601.5.4.51',
 '503.5.7.4',
 '1065.5.4.38',
 '923.3.4.17',
 '166.4.6.0',
 '184.5.7.47',
 '299.3.6.16',
 '823.3.7.45',
 '362.3.5.15',
 '659.3.4.32'

In [42]:
subset_df['glimpse_ID'] = None
for index, row in tqdm(subset_df.iterrows(), total = len(subset_df)):
    unique_ID = row['Unique ID']
    time = row['Time (hours)']
    glimpse_ID = f'{unique_ID}.{time}'
    subset_df.at[index, 'glimpse_ID'] = glimpse_ID

  0%|          | 0/20900 [00:00<?, ?it/s]

In [43]:
subset_df

,Time (hours),x,y,x scaled,y scaled,Infection status,Initial infection status,Final infection status,Area,Intracellular mean Mtb content,...,MSD,Strain,Compound,Concentration,Cell ID,Acquisition ID,Unique ID,Technical replicate,dMtb,glimpse_ID
0,0,708.125061,77.589088,3568.950308,391.049006,0.0,0.0,1.0,42556.0,365.408905,...,0.000000,RD1,CTRL,EC0,433,"(3, 4)",433.3.4,1.0,0.095232,433.3.4.0
1,1,725.449097,73.245003,3656.263447,369.154814,0.0,0.0,1.0,48727.0,362.852478,...,17.860383,RD1,CTRL,EC0,433,"(3, 4)",433.3.4,1.0,0.095232,433.3.4.1
2,2,723.088928,76.431496,3644.368198,385.214738,0.0,0.0,1.0,43484.0,363.515442,...,3.965367,RD1,CTRL,EC0,433,"(3, 4)",433.3.4,1.0,0.095232,433.3.4.2
3,3,726.009766,79.395073,3659.089219,400.151168,0.0,0.0,1.0,39113.0,360.448578,...,4.161019,RD1,CTRL,EC0,433,"(3, 4)",433.3.4,1.0,0.095232,433.3.4.3
4,4,728.209656,82.197769,3670.176665,414.276757,0.0,0.0,1.0,32644.0,362.289764,...,3.562951,RD1,CTRL,EC0,433,"(3, 4)",433.3.4,1.0,0.095232,433.3.4.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
697433,70,565.650940,1081.278931,2850.880737,5449.645811,0.0,1.0,0.0,61587.0,374.370483,...,0.963656,WT,BDQ,EC99,1202,"(6, 9)",1202.6.9,2.0,-0.835031,1202.6.9.70
697434,71,566.611755,1083.470947,2855.723247,5460.693574,0.0,1.0,0.0,58778.0,372.559570,...,2.393346,WT,BDQ,EC99,1202,"(6, 9)",1202.6.9,2.0,-0.835031,1202.6.9.71
697435,72,568.148987,1081.411011,2863.470894,5450.311494,0.0,1.0,0.0,64893.0,372.814087,...,2.570296,WT,BDQ,EC99,1202,"(6, 9)",1202.6.9,2.0,-0.835031,1202.6.9.72
697436,73,570.763306,1084.552124,2876.647061,5466.142705,1.0,1.0,0.0,61917.0,374.565948,...,4.086717,WT,BDQ,EC99,1202,"(6, 9)",1202.6.9,2.0,-0.835031,1202.6.9.73


In [45]:
subset_df = subset_df[~subset_df['glimpse_ID'].isin(already_made_glimpses)]

In [46]:
# Define your cropping and saving function
def crop_and_save(unique_ID):
    
    # define the previoulsy loaded row and column as global so that they are remebered outside function
    global previous_row, previous_column, non_upload_list
    
    # get cell and position ID from UID
    cell_ID, row, column = unique_ID.split('.') 
    
    # get all time values for specific cell 
    time = subset_df[subset_df['Unique ID'] == unique_ID]['Time (hours)'].values
    
    # create a list of time values to iterate over 
    time_to_do = list()
    
    # see if any of the glimpses have been made (do this here to avoid loading seg unless there is time to do)
    for t in tqdm(time, desc = f'Iterating over {len(time)} time points in cell ID {unique_ID}', leave = False):
        
        # make t an int
        t = int(t)
        
        # format output fn
        output_fn = f'/mnt/DATA/macrohet/results/glimpses/interactive_plots/time_series/glimpse_{cell_ID}.{row}.{column}_t{t}.png'
        
        # check to see if exists 
        if not os.path.exists(output_fn):
            
            # if it doesnt exist then add it to new list of time points to 
            time_to_do.append(t)
    
    # if there are no timepoints to make then continue to next cell
    if time_to_do == None:
        return
            
    # check to see if previous position ID is the same to avoid reloading segmentaion 
    if (row, column) != (previous_row, previous_column):
        
        # if it is new then reload
        with btrack.io.HDF5FileHandler(os.path.join(base_dir,
                                                    f'labels/macrohet_seg_model/{int(row),int(column)}.h5'),
                                       'r',
                                       obj_type='obj_type_1') as reader:
            
            # get segmentation 
            segmentation = reader.segmentation
            
    # then iterate over all the time points that don't have a glimpse already made
    for t in tqdm(time, desc = f'Iterating over {len(time)} time points in cell ID {unique_ID}', leave = False):
        
        # again turn into int
        t = int(t)
        
        # create output path 
        output_fn = f'/mnt/DATA/macrohet/results/glimpses/interactive_plots/time_series/glimpse_{cell_ID}.{row}.{column}_t{t}.png'

        # create glimpse
        glimpse = visualise.tn_glimpse_maker(unique_ID, 
                                             df, 
                                             time=t, 
                                             metadata=metadata, 
                                             segmentation=segmentation)
        # save glimpse out 
        imsave(output_fn, glimpse)
        
        # extract basename from file for aws upload
        basename = os.path.basename(output_fn)
        
        # create aws fn
        aws_fn = f'time_series/all_glimpses/{basename}'
        
        # try uploading 
        try:
            aws_tools.upload(output_fn, 
                             'macrohet', 
                                aws_fn, output_msg = False)
        # if it doesnt work make note of it
        except:
            non_upload_list.append(basename)
            
    # define previous row and column
    previous_row, previous_column = row, column

# Define the number of processes to run in parallel
num_processes = int(multiprocessing.cpu_count() - 10)  # Use the number of available CPU cores

# Create a Pool of worker processes
pool = multiprocessing.Pool(processes=num_processes)


[INFO][2023/06/29 01:40:26 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 8).h5...
[INFO][2023/06/29 01:40:26 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 8).h5...
[INFO][2023/06/29 01:40:26 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(4, 4).h5...
[INFO][2023/06/29 01:40:26 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 9).h5...
[INFO][2023/06/29 01:40:26 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 7).h5...
[INFO][2023/06/29 01:40:26 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 9).h5...
[INFO][2023/06/29 01:40:26 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(4, 7).h5...
[INFO][2023/06/29 01:40:26 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 7).h5...
[INFO][2023/06/29 01:40:26 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 9).h5...
[INFO][2023/06/29 01:40:26 PM] Openin

Iterating over 57 time points in cell ID 1365.3.7:   0%|          | 0/57 [00:00<?, ?it/s]

[INFO][2023/06/29 01:40:43 PM] Loading segmentation (75, 6048, 6048)
[INFO][2023/06/29 01:40:43 PM] Closing HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(4, 4).h5


Iterating over 75 time points in cell ID 288.4.4:   0%|          | 0/75 [00:00<?, ?it/s]

[INFO][2023/06/29 01:40:44 PM] Loading segmentation (75, 6048, 6048)
[INFO][2023/06/29 01:40:44 PM] Closing HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(4, 5).h5


...

Iterating over 58 time points in cell ID 1494.4.8:   0%|          | 0/58 [00:00<?, ?it/s]

Iterating over 43 time points in cell ID 1805.4.6:   0%|          | 0/43 [00:00<?, ?it/s]

Iterating over 74 time points in cell ID 461.4.8:   0%|          | 0/74 [00:00<?, ?it/s]

[INFO][2023/06/29 01:41:17 PM] Loading segmentation (75, 6048, 6048)
[INFO][2023/06/29 01:41:17 PM] Closing HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 9).h5


Iterating over 72 time points in cell ID 696.3.9:   0%|          | 0/72 [00:00<?, ?it/s]

In [47]:
num_processes

26

In [ ]:

# Apply the crop_and_save function to each image in parallel
results = pool.map(crop_and_save, subset_df['Unique ID'].unique())

# Close the pool to prevent any more tasks from being submitted
pool.close()

# Wait for all processes to complete
pool.join()

# Process the results if needed

# Put a wrapper around it to time execution

In [ ]:
import tqdm

# Get the unique IDs
unique_IDs = subset_df['Unique ID'].unique()

# Create a pool of worker processes
pool = multiprocessing.Pool(processes=num_processes)

# Define the progress bar
progress_bar = tqdm.tqdm(total=len(unique_IDs), desc="Processing Images")

# Function to track progress
def update_progress(*args):
    progress_bar.update()

# Apply the crop_and_save function to each unique ID in parallel
results = pool.map_async(crop_and_save, unique_IDs, callback=update_progress)

# Wait for all processes to complete
results.wait()

# Close the pool
pool.close()
pool.join()

# Close the progress bar
progress_bar.close()


In [ ]:
send_sms('Done')

In [ ]:
import time
for i in range(0,14):
    time.sleep(60*60)
    send_sms('Kernel still standing after all this time')